<a href="https://colab.research.google.com/github/Dr-Carlos-Villasenor/DIP/blob/main/DIP_041.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Análisis y procesamiento de imágenes
## Dr. Carlos Villaseñor
### DIP_041 - Transferencia de aprendizaje

Importar bibliotecas



In [23]:
import pandas as pd
import numpy as np
import os
import keras
import matplotlib.pyplot as plt
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import mobilenet
from keras.preprocessing import image
from keras.applications.mobilenet import preprocess_input
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint
from keras.models import Model

Para subir los archivos tenemos dos opciones, cargar su carpeta de drive:

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Cargar un archivo .zip y descomprimirlo

- Carge el archivo 'TL_dataset.zip' en la pestaña archivos en el lateral izquierdo
- Corra la siguiente linea para descomprimir los archivos

In [ ]:
!unzip TL_dataset.zip

Importar MobileNet base pero descartaremos las últimas capa

In [ ]:
base_model=mobilenet.MobileNet(weights='imagenet',include_top=False)
print(base_model.summary())

Crear nueva Red Neuronal

In [25]:
x=base_model.output
x=GlobalAveragePooling2D()(x)
x=Dense(64,activation='relu')(x) 
x=Dense(32,activation='relu')(x) 
preds=Dense(3, activation='softmax')(x)

Pegar Redes Neuronales

In [ ]:
model=Model(inputs=base_model.input, outputs=preds)
print(model.summary())

Definir que capas se entrenan

In [27]:
for layer in model.layers[:-5]:
    layer.trainable=False
for layer in model.layers[-5:]:
    layer.trainable=True

Crear generador desde carpeta

In [ ]:
train_datagen=ImageDataGenerator(zoom_range=0.2, horizontal_flip=True,
                                 width_shift_range=0.2,
                                 height_shift_range=0.2,
                                 preprocessing_function=preprocess_input)

train_generator=train_datagen.flow_from_directory('./TL_dataset/train/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=32,
                                                 class_mode='categorical',
                                                 shuffle=True)

test_datagen=ImageDataGenerator(preprocessing_function=preprocess_input)

test_generator=test_datagen.flow_from_directory('./TL_dataset/test/',
                                                 target_size=(224,224),
                                                 color_mode='rgb',
                                                 batch_size=31,
                                                 class_mode='categorical',
                                                 shuffle=False)



Compilar Modelo y entrenar desde carpeta

In [ ]:
model.compile(optimizer='Adam',loss='categorical_crossentropy',
              metrics=['accuracy'])

step_size_train=train_generator.n//train_generator.batch_size

filepath="weights-improvement-{epoch:02d}-{accuracy:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='accuracy',
                             save_weights_only=False,
                             verbose=1, save_best_only=True, mode='max')

model.fit_generator(generator=train_generator,
                    steps_per_epoch=step_size_train,
                    epochs=6,  callbacks=[checkpoint])

Evaluación final del modelo

In [ ]:
step_size_test = test_generator.n // test_generator.batch_size
model.evaluate(test_generator, steps=step_size_test)

In [ ]:
from keras.models import load_model
model = load_model('./model.hdf5')

test_datagen = ImageDataGenerator(preprocessing_function=preprocess_input)
test_generator = test_datagen.flow_from_directory('./TL_dataset/test/',
                                                  target_size=(224, 224),
                                                  color_mode='rgb',
                                                  batch_size=1,
                                                  class_mode='categorical',
                                                  shuffle=True)

In [ ]:
import matplotlib.pyplot as plt 
image, classe = next(test_generator)
classes = np.argmax(classe, axis=-1)
pred = np.argmax(model.predict(image), axis=1)

names = {0:'cat', 1:'dog', 2:'horse'}

plt.imshow(image[0])
plt.title('Network pred: ' + names[pred[0]])